In [1]:
import multiprocessing
import time
import numpy as np

<p>I have a function sillySquare that I use to compute the square of a bunch of numbers. Unfortunately I also write silly code and my function takes way too long to compute for large datasets. I supose I could just write a more efficient function for computing the square, but that is not always possible and besides, would be no fun.</p>
<p>Computing the square of a bunch of numbers is an example of an "embarrassingly parallel problem". The problem is parallel because the result of computing one square does not influence the result of any other subsequent calculation. The problem is embarrassing because....They are called that; embarrassingly parallel problems are <i><b>nothing</b></i> to be ashamed of, in fact as we will see, they are a great kind of problem to have!</p>

In [2]:
def sillySquare( x, waitTime ):
    """
    sillySquare takes aproximatly waitTime seconds to do the
    hard work of computing x*x.
    
    INPUTS:
        x        -- (float/int) Value to square.
        waitTime -- (float/int) Number of seconds to waste.
    
    RETURN:    
         (float) value of x squared 
    """
    time.sleep(waitTime)
    return x*x

<p>To show that our function is slow and silly lets use it to compute the square of a dataset of eight values. If we set waitTime to 1 second our work should take approximately 8 seconds to complete.</p>

In [3]:
our_data  = np.random.randint(0,10,8)
wait_time = np.ones(8, dtype=np.int) 
print('Our data is    : {}'.format(our_data))
print('Our waitTime is: {}'.format(wait_time))

Our data is    : [3 8 0 0 0 4 5 8]
Our waitTime is: [1 1 1 1 1 1 1 1]


In [4]:
t_start = time.time()

results = []

for x,wait in zip(our_data,wait_time):
    square = sillySquare(x, wait)
    results.append(square)

t_end = time.time()

print("Job took {:0.3} seconds".format(t_end-t_start))
print("Computed the values: {}".format(results))

Job took 8.02 seconds
Computed the values: [9, 64, 0, 0, 0, 16, 25, 64]


<p>In our for loop above all the work was being done by a single CPU in series. Work on the first value in our dataset had to finish before work on the second value could begin. Modern computers have many logical CPU's however; we could be using them all to speed this calculation up!</p>
<p>We can do this using the multiprocessing library. (Note: multiprocessing is not the same thing as multithreading, another term you may be familiar with. Do not confuse the two, they are actually quite different.)</p> 

In [5]:
n_cpus = multiprocessing.cpu_count()

print("This machine has {0} logical cpu's.\
 Lets use them to speed things up!".format(n_cpus))

This machine has 8 logical cpu's. Lets use them to speed things up!


In [6]:
t_start = time.time()

#Create a job pool using the multiprocessing library
job_pool = multiprocessing.Pool(n_cpus)

#Apply the job pool to our function handle 
# and an iterable of input params [(x0, waitTime0) ... (x7, waitTime7)].
parallel_square = job_pool.starmap(sillySquare, zip(our_data,wait_time) )

#close our job pool
job_pool.close()

#Enjoy the 7 seconds we just saved!

t_end = time.time()
print("Job took {:0.3} seconds".format(t_end-t_start))
print("Computed the values: {}".format(parallel_square))

Job took 1.04 seconds
Computed the values: [9, 64, 0, 0, 0, 16, 25, 64]


<p>Nice!</p>